In [4]:
import pandas as pd
import requests
from tqdm import tqdm

CSV_IN = 'e약은요.csv'
TXT_OUT = 'e약은요_요약본.txt'
PROGRESS_FILE = 'progress.txt'  # 진행상황 저장용

def call_llama(prompt, max_tokens=1024):
    url = 'http://localhost:11434/api/generate'
    headers = {'Content-Type': 'application/json'}
    payload = {
        'model': 'llama3.1:8b',
        'prompt': prompt,
        'stream': False,
        'options': {
            'num_predict': max_tokens,
            'temperature': 0.7
        }
    }
    res = requests.post(url, headers=headers, json=payload, timeout=180)
    res.raise_for_status()
    response = res.json()
    return response['response'].strip()

def process_single_row(row):
    """CSV의 모든 컬럼을 적절히 처리"""
    raw_text = ','.join(map(str, row.values))
    columns = raw_text.split(',')
    
    # 1. 제품명 - 그대로
    product_name = columns[0] if len(columns) > 0 else "정보없음"
    
    # 2. 제약사 - 그대로  
    manufacturer = columns[1] if len(columns) > 1 else "정보없음"
    
    # 3. 주성분 - 깔끔하게 정렬
    ingredients_raw = columns[2] if len(columns) > 2 else ""
    ingredients_prompt = f"""다음 주성분 정보를 보기 좋게 정리해줘. 각 성분을 콤마로 구분하고, 복잡한 화학명은 괄호 안에 넣어서:

{ingredients_raw}

예시: 아세트아미노펜, 카페인무수물, 이소프로필안티피린"""
    try:
        ingredients = call_llama(ingredients_prompt, max_tokens=256)
    except Exception as e:
        ingredients = f"처리오류: {e}"
    
    # 4. 효능 및 효과 - 내용을 파악해서 적절히 분류
    efficacy_raw = columns[3] if len(columns) > 3 else ""
    efficacy_prompt = f"""다음 의약품의 효능 및 효과를 읽고, 이 약이 어떤 용도인지 파악해서 보기 좋게 정리해줘.

원문: {efficacy_raw}

다음과 같이 정리해줘:
1. 이 약의 주요 용도가 무엇인지 파악 (예: 해열진통제, 항생제, 소화제, 감기약 등)."""
    try:
        efficacy = call_llama(efficacy_prompt, max_tokens=1024)
    except Exception as e:
        efficacy = f"처리오류: {e}"
    
    # 5. 용법 및 용량 - 이해하기 쉽게
    dosage_raw = columns[4] if len(columns) > 4 else ""
    dosage_prompt = f"""다음 용법 및 용량 정보를 보기 쉽게 정리해줘:

{dosage_raw}

다음과 같이 나누어서:
- 복용량: 
- 복용횟수:
- 복용방법:
- 주의사항:"""
    try:
        dosage = call_llama(dosage_prompt, max_tokens=512)
    except Exception as e:
        dosage = f"처리오류: {e}"
    
    # 6. 금기사항 - 명확하게 분류
    contraindications_raw = columns[5] if len(columns) > 5 else ""
    contraindications_prompt = f"""다음 금기사항을 사용자를 위한 약 설명서 형식으로 보기 쉽게 분류해서 정리해 주세요. 
    질환별, 연령별, 약물상호작용별로 나누어 bullet point로 하되, 너무 세분화하지 말고 읽기 쉽게 하세요. 
    분류 제목을 적절히 붙이고, 설명은 자연스럽고 명확하게 하세요. 출력은 정리된 내용만 하세요. 프롬프트 지시나 번호를 붙이지 말고, 바로 본문부터 시작하세요.

    {contraindications_raw}"""
    try:
        contraindications = call_llama(contraindications_prompt, max_tokens=768)
    except Exception as e:
        contraindications = f"처리오류: {e}"
        
    # 7. 주의사항 - 상황별로 분류 (개선: '설명서' 스타일 강조)
    precautions_raw = columns[6] if len(columns) > 6 else ""
    precautions_prompt = f"""다음 주의사항을 사용자를 위한 약 설명서 형식으로 상황별로 분류해서 정리해 주세요. 
    특정 질환자, 임신/수유부, 고령자 등으로 나누어 bullet point로 하되, 너무 세분화하지 말고 읽기 쉽게 하세요. 
    분류 제목을 적절히 붙이고, 설명은 자연스럽고 명확하게 하세요. 출력은 정리된 내용만 하세요. 프롬프트 지시나 번호를 붙이지 말고, 바로 본문부터 시작하세요.

    {precautions_raw}"""
    try:
        precautions = call_llama(precautions_prompt, max_tokens=768)
    except Exception as e:
        precautions = f"처리오류: {e}"
        
    # 8. 병용금기 - 약물별로 정리 (개선: '설명서' 스타일 강조)
    interactions_raw = columns[7] if len(columns) > 7 else ""
    interactions_prompt = f"""다음 병용금기 정보를 사용자를 위한 약 설명서 형식으로 보기 쉽게 정리해 주세요. 
    절대 금기약물과 주의약물로 나누어 bullet point로 하되, 읽기 쉽게 하세요. 
    분류 제목을 적절히 붙이고, 설명은 자연스럽고 명확하게 하세요. 출력은 정리된 내용만 하세요. 프롬프트 지시나 번호를 붙이지 말고, 바로 본문부터 시작하세요.

    {interactions_raw}"""
    try:
        interactions = call_llama(interactions_prompt, max_tokens=512)
    except Exception as e:
        interactions = f"처리오류: {e}"
        
    # 9. 부작용 - 심각도별로 분류 (개선: '설명서' 스타일 강조)
    side_effects_raw = columns[8] if len(columns) > 8 else ""
    side_effects_prompt = f"""다음 부작용 정보를 사용자를 위한 약 설명서 형식으로 심각도별로 분류해서 정리해 주세요. 
    즉시 중단해야 할 심각한 부작용과 일반적인 부작용으로 나누어 bullet point로 하되, 읽기 쉽게 하세요. 
    분류 제목을 적절히 붙이고, 설명은 자연스럽고 명확하게 하세요. 출력은 정리된 내용만 하세요. 프롬프트 지시나 번호를 붙이지 말고, 바로 본문부터 시작하세요.

    {side_effects_raw}"""
    try:
        side_effects = call_llama(side_effects_prompt, max_tokens=768)
    except Exception as e:
        side_effects = f"처리오류: {e}"
        
    # 10. 보관방법 - 간단하게 (개선: '설명서' 스타일 강조)
    storage_raw = columns[9] if len(columns) > 9 else ""
    storage_prompt = f"""다음 보관방법을 사용자를 위한 약 설명서 형식으로 간단명료하게 정리해 주세요. 
    bullet point로 하되, 읽기 쉽게 하세요. 출력은 정리된 내용만 하세요. 프롬프트 지시나 번호를 붙이지 말고, 바로 본문부터 시작하세요.

    {storage_raw}"""
    try:
        storage = call_llama(storage_prompt, max_tokens=256)
    except Exception as e:
        storage = f"처리오류: {e}"
        
    # 최종 결과 조합 (원본과 동일)
    result = f"""제품명: {product_name}
    제약사: {manufacturer}
    주성분: {ingredients}

    효능 및 효과:
    {efficacy}

    용법 및 용량:
    {dosage}

    복용하면 안 되는 경우 (금기):
    {contraindications}

    복용 전 의사/약사와 상담이 필요한 경우:
    {precautions}

    함께 복용하면 안 되는 약물:
    {interactions}

    주요 부작용:
    {side_effects}

    보관방법:
    {storage}

    {'='*80}

    """
    
    return result

def get_last_processed():
    """마지막으로 처리한 행 번호 가져오기"""
    try:
        with open(PROGRESS_FILE, 'r', encoding='utf-8') as f:
            return int(f.read().strip())
    except:
        return -1

def save_progress(row_num):
    """진행상황 저장"""
    with open(PROGRESS_FILE, 'w', encoding='utf-8') as f:
        f.write(str(row_num))

# 메인 실행 부분 - 텍스트 파일 저장
df = pd.read_csv(CSV_IN)
last_processed = get_last_processed()

with open(TXT_OUT, 'a', encoding='utf-8') as f:  # 'a'는 append 모드
    for i, row in tqdm(df.iterrows(), total=len(df), desc="의약품 정보 정리"):
        # 이미 처리된 행은 건너뛰기
        if i <= last_processed:
            continue
        
        try:
            summary = process_single_row(row)
            f.write(summary)  # 텍스트 파일에 바로 추가
            f.flush()  # 즉시 디스크에 저장
            save_progress(i)  # 진행상황 저장
            tqdm.write(f'{i+1}/{len(df)}행 완료 및 저장')
        except Exception as e:
            error_msg = f'처리 오류 (행 {i+1}): {e}\n{"="*80}\n'
            f.write(error_msg)
            tqdm.write(f'오류 발생: 행 {i+1} - {e}')

print('텍스트 파일 저장 완료!')


의약품 정보 정리:   0%|          | 7/4766 [01:13<13:58:02, 10.57s/it]

7/4766행 완료 및 저장


의약품 정보 정리:   0%|          | 7/4766 [02:08<24:11:02, 18.29s/it]


KeyboardInterrupt: 